**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Production economy and CO2 taxation](#toc1_)    
- 2. [Problem 2: Career choice model](#toc2_)    
- 3. [Problem 3: Barycentric interpolation](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [3]:
import numpy as np
from types import SimpleNamespace
from scipy.optimize import minimize_scalar, root

## 1. <a id='toc1_'></a>[Problem 1: Production economy and CO2 taxation](#toc0_)

Consider a production economy with two firms indexed by $j \in \{1,2\}$. Each produce its own good. They solve

$$
\begin{align*}
\max_{y_{j}}\pi_{j}&=p_{j}y_{j}-w_{j}\ell_{j}\\\text{s.t.}\;&y_{j}=A\ell_{j}^{\gamma}.
\end{align*}
$$

Optimal firm behavior is

$$
\begin{align*}
\ell_{j}^{\star}(w,p_{j})&=\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}} \\
y_{j}^{\star}(w,p_{j})&=A\left(\ell_{j}^{\star}(w,p_{j})\right)^{\gamma}
\end{align*}
$$

The implied profits are

$$
\pi_{j}^*(w,p_{j})=\frac{1-\gamma}{\gamma}w\cdot\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}}
$$

A single consumer supplies labor, and consumes the goods the firms produce. She also recieves the implied profits of the firm.<br>
She solves:

$$
\begin{align*}
U(p_1,p_2,w,\tau,T) = \max_{c_{1},c_{2},\ell} & \log(c_{1}^{\alpha}c_{2}^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} \\
\text{s.t.}\,\,\,&p_{1}c_{1}+(p_{2}+\tau)c_{2}=w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})
\end{align*}
$$

where $\tau$ is a tax and $T$ is lump-sum transfer. <br>
For a given $\ell$, it can be shown that optimal behavior is

$$
\begin{align*}
c_{1}(\ell)&=\alpha\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{1}} \\
c_{2}(\ell)&=(1-\alpha)\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{2}+\tau} \\
\end{align*}
$$
Such that optimal behavior is:
$$
\ell^* = \underset{\ell}{\arg\max} \log(\left(c_{1}(\ell)\right)^{\alpha}\cdot \left(c_{2}(\ell)\right)^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} 
$$
With optimal consumption:
$$
\begin{align*}
c_1^*=c_{1}(\ell^*) \\
c_2^*=c_{2}(\ell^*)\\
\end{align*}
$$


The government chooses $\tau$ and balances its budget so $T=\tau c_2^*$. We initially set $\tau,T=0$.

Market clearing requires:

1. Labor market: $\ell^* = \ell_1^* + \ell_2^*$
1. Good market 1: $c_1^* = y_1^*$
1. Good market 2: $c_2^* = y_2^*$


**Question 1:** Check market clearing conditions for $p_1$ in `linspace(0.1,2.0,10)` and $p_2$ in `linspace(0.1,2.0,10)`. We choose $w=1$ as numeraire.

In [4]:
# Setting parameters
par = SimpleNamespace()

# Firms
par.A = 1.0
par.gamma = 0.5

# Households
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0

# Government
par.tau = 0.0
par.T = 0.0

# Question 3
par.kappa = 0.1

In [5]:
# Firm functions
def labor_demand(p):
    return (p * par.A * par.gamma) ** (1 / (1 - par.gamma))

def good_supply(l):
    return par.A * l ** par.gamma

def profit(p):
    return ((1 - par.gamma) / par.gamma) * (p * par.A * par.gamma) ** (1 / (1 - par.gamma))

# Consumer functions
def good_demand_1(p1, p2, l):
    income = l + par.T + profit(p1) + profit(p2)
    return par.alpha * income / p1

def good_demand_2(p1, p2, l):
    income = l + par.T + profit(p1) + profit(p2)
    return (1 - par.alpha) * income / (p2 + par.tau)

def labor_supply(p1, p2):
    # Define the objective function
    def objective(l):
        c1 = good_demand_1(p1, p2, l)
        c2 = good_demand_2(p1, p2, l)
        return np.log(c1 ** par.alpha * c2 ** (1 - par.alpha)) - par.nu * l ** (1 + par.epsilon) / (1 + par.epsilon)
    
    # Find the labor that maximizes the objective
    result = minimize_scalar(lambda l: -objective(l), bounds=(0, 10), method='bounded')
    return result.x

# Check market clearing conditions
for p1 in np.linspace(0.1, 2.0, 10):
    for p2 in np.linspace(0.1, 2.0, 10):
        # Firm labor demand
        labor_demand_1 = labor_demand(p1)
        labor_demand_2 = labor_demand(p2)

        # Firm good supply
        good_supply_1 = good_supply(labor_demand_1)
        good_supply_2 = good_supply(labor_demand_2)

        # Consumer labor supply
        l = labor_supply(p1, p2)

        # Consumer good demand
        c1 = good_demand_1(p1, p2, l)
        c2 = good_demand_2(p1, p2, l)

        # Market clearing conditions
        LM = l - labor_demand_1 - labor_demand_2
        GM1 = good_supply_1 - c1
        GM2 = good_supply_2 - c2

        # printing results
        print(f'(p1,p2) = ({p1:.2f},{p2:.2f}) -> Excess supply on Labor Market: {LM:.2f}, Good Market 1: {GM1:.2f}, Good Market 2: {GM2:.2f}')

(p1,p2) = (0.10,0.10) -> Excess supply on Labor Market: 0.99, Good Market 1: -2.96, Good Market 2: -6.97
(p1,p2) = (0.10,0.31) -> Excess supply on Labor Market: 0.96, Good Market 1: -3.00, Good Market 2: -2.13
(p1,p2) = (0.10,0.52) -> Excess supply on Labor Market: 0.91, Good Market 1: -3.09, Good Market 2: -1.14
(p1,p2) = (0.10,0.73) -> Excess supply on Labor Market: 0.82, Good Market 1: -3.23, Good Market 2: -0.68
(p1,p2) = (0.10,0.94) -> Excess supply on Labor Market: 0.70, Good Market 1: -3.42, Good Market 2: -0.38
(p1,p2) = (0.10,1.16) -> Excess supply on Labor Market: 0.56, Good Market 1: -3.66, Good Market 2: -0.17
(p1,p2) = (0.10,1.37) -> Excess supply on Labor Market: 0.40, Good Market 1: -3.95, Good Market 2: -0.00
(p1,p2) = (0.10,1.58) -> Excess supply on Labor Market: 0.20, Good Market 1: -4.31, Good Market 2: 0.14
(p1,p2) = (0.10,1.79) -> Excess supply on Labor Market: -0.01, Good Market 1: -4.73, Good Market 2: 0.27
(p1,p2) = (0.10,2.00) -> Excess supply on Labor Market: 

**Question 2:** Find the equilibrium prices $p_1$ and $p_2$.<br>
*Hint: you can use Walras' law to only check 2 of the market clearings*

In [6]:
# Define function that expresses excess demand given prices
def market_clearing(prices):
    p1, p2 = prices
    labor_demand_1 = labor_demand(p1)
    labor_demand_2 = labor_demand(p2)
    good_supply_1 = good_supply(labor_demand_1)
    l = labor_supply(p1, p2)
    c1 = good_demand_1(p1, p2, l)
    LM = l - labor_demand_1 - labor_demand_2
    GM1 = good_supply_1 - c1
    return [LM, GM1]

# Finding equilibrium prices, with (p1,p2) = (1.0,1.0) as initial guess
result = root(market_clearing, [1.0, 1.0])

# Check if solution was found
if result.success:
    equilibrium_prices = result.x
    print(f'Equilibrium prices: p1 = {equilibrium_prices[0]:.2f}, p2 = {equilibrium_prices[1]:.2f}')
else:
    print('Equilibrium not found')

Equilibrium prices: p1 = 0.98, p2 = 1.49


Assume the government care about the social welfare function:

$$
SWF = U - \kappa y_2^*
$$

Here $\kappa$ measures the social cost of carbon emitted by the production of $y_2$ in equilibrium.

**Question 3:** What values of $\tau$ and (implied) $T$ should the government choose to maximize $SWF$?

In [7]:
# Function to find equilibrium for given tau
def find_equilibrium(tau, T):
    par.tau = tau
    par.T = T

    initial_prices = [1.0, 1.0]
    result = root(market_clearing, initial_prices)

    if result.success:
        optimal_prices = result.x
        l = labor_supply(optimal_prices[0], optimal_prices[1])
        c2 = good_demand_2(optimal_prices[0], optimal_prices[1], l)
        return c2, optimal_prices
    else:
        raise ValueError('Equilibrium prices not found')

# Social welfare function
def social_welfare(tau):
    # Initial guess for T
    T = 0
    tolerance = 1e-6
    max_iterations = 100
    iteration = 0
    
    while iteration < max_iterations:
        iteration += 1
        previous_T = T
        c2, optimal_prices = find_equilibrium(tau, T)
        T = tau * c2
        
        if abs(T - previous_T) < tolerance:
            break

    # Final equilibrium with converged T
    c2, optimal_prices = find_equilibrium(tau, T)
    l = labor_supply(optimal_prices[0], optimal_prices[1])
    c1 = good_demand_1(optimal_prices[0], optimal_prices[1], l)
    
    # Utility function
    U = np.log(c1 ** par.alpha * c2 ** (1 - par.alpha)) - par.nu * l ** (1 + par.epsilon) / (1 + par.epsilon)
    # Social welfare function
    SWF = U - par.kappa * c2
    return -SWF  # We minimize the negative of SWF to maximize SWF

# Finding optimal tau
result_tau = minimize_scalar(social_welfare, bounds=(0, 1), method='bounded')

# Check if solution was found
if result_tau.success:
    optimal_tau = result_tau.x
    print(f'Optimal tau is {optimal_tau:.2f}, which leads to SWF = {-result_tau.fun:.2f}')
else:
    print('Optimal tau not found')

Optimal tau is 0.17, which leads to SWF = -0.66


## 2. <a id='toc2_'></a>[Problem 2: Career choice model](#toc0_)

Consider a graduate $i$ making a choice between entering $J$ different career tracks. <br>
Entering career $j$ yields utility $u^k_{ij}$. This value is unknown to the graduate ex ante, but will ex post be: <br>
$$
    u_{i,j}^k = v_{j} + \epsilon_{i,j}^k
$$

They know that $\epsilon^k_{i,j}\sim \mathcal{N}(0,\sigma^2)$, but they do not observe $\epsilon^k_{i,j}$ before making their career choice. <br>

Consider the concrete case of $J=3$ with:
$$
\begin{align*}
    v_{1} &= 1 \\
    v_{2} &= 2 \\
    v_{3} &= 3
\end{align*}
$$

If the graduates know the values of $v_j$ and the distribution of $\epsilon_{i,j}^k$, they can calculate the expected utility of each career track using simulation: <br>
$$
    \mathbb{E}\left[ u^k_{i,j}\vert v_j \right] \approx v_j + \frac{1}{K}\sum_{k=1}^K \epsilon_{i,j}^k
$$

In [47]:
par = SimpleNamespace()
par.J = 3
par.N = 10
par.K = 10000

par.F = np.arange(1,par.N+1)
par.sigma = 2

par.v = np.array([1,2,3])
par.c = 1

**Question 1:** Simulate and calculate expected utility and the average realised utility for $K=10000$ draws, for each career choice $j$.


In [48]:
# write your answer here

Now consider a new scenario: Imagine that the graduate does not know $v_j$. The *only* prior information they have on the value of each job, comes from their $F_{i}$ friends that work in each career $j$. After talking with them, they know the average utility of their friends (which includes their friends' noise term), giving them the prior expecation: <br>
$$
\tilde{u}^k_{i,j}\left( F_{i}\right) = \frac{1}{F_{i}}\sum_{f=1}^{F_{i}} \left(v_{j} + \epsilon^k_{f,j}\right), \; \epsilon^k_{f,j}\sim \mathcal{N}(0,\sigma^2)
$$
For ease of notation consider that each graduate have $F_{i}=i$ friends in each career. <br>

For $K$ times do the following: <br>
1. For each person $i$ draw $J\cdot F_i$ values of $\epsilon_{f,j}^{k}$, and calculate the prior expected utility of each career track, $\tilde{u}^k_{i,j}\left( F_{i}\right)$. <br>
Also draw their own $J$ noise terms, $\epsilon_{i,j}^k$
1. Each person $i$ chooses the career track with the highest expected utility: $$j_i^{k*}= \arg\max_{j\in{1,2\dots,J}}\left\{ \tilde{u}^k_{i,j}\left( F_{i}\right)\right\} $$
1. Store the chosen careers: $j_i^{k*}$, the prior expectation of the value of their chosen career: $\tilde{u}^k_{i,j=j_i^{k*}}\left( F_{i}\right)$, and the realized value of their chosen career track: $u^k_{i,j=j_i^{k*}}=v_{j=j_i^{k*}}+\epsilon_{i,j=j_i^{k*}}^k$.

Chosen values will be: <br>
$i\in\left\{1,2\dots,N\right\}, N=10$ <br>
$F_i = i$<br>
So there are 10 graduates. The first has 1 friend in each career, the second has 2 friends, ... the tenth has 10 friends.

**Question 2:** Simulate and visualize: For each type of graduate, $i$, the share of graduates choosing each career, the average subjective expected utility of the graduates, and the average ex post realized utility given their choice. <br>
That is, calculate and visualize: <br>
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} \mathbb{I}\left\{ j=j_i^{k*} \right\}  \;\forall j\in\left\{1,2,\dots,J\right\}
\end{align*}
$$
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} \tilde{u}^k_{ij=j_i^{k*}}\left( F_{i}\right)
\end{align*}
$$
And 
$$
\begin{align*}
    \frac{1}{K} \sum_{k=1}^{K} u^k_{ij=j_i^{k*}} 
\end{align*}
$$
For each graduate $i$.

In [49]:
# Write your answer here 

After a year of working in their career, the graduates learn $u^k_{ij}$ for their chosen job $j_i^{k*}$ perfectly. <br>
The can switch to one of the two remaining careers, for which they have the same prior as before, but it will now include a switching cost of $c$ which is known.
Their new priors can be written as: 
$$
\tilde{u}^{k,2}_{ij}\left( F_{i}\right) = \begin{cases}
            \tilde{u}^k_{ij}\left( F_{i}\right)-c & \text{if } j \neq j_i^{k*} \\
            u_{ij=j_i^{k*}} & \text{if } j = j_i^{k*}
        \end{cases}
$$

We will set $c=1$.

Their realized utility will be: <br>
$$
u^{k,2}_{ij}= \begin{cases}
            u_{ij}^k -c & \text{if } j \neq j_i^{k*} \\
            u_{ij=j_i^{k*}} & \text{if } j = j_i^{k*}
        \end{cases}
$$

**Question 3:** Following the same approach as in question 2, find the new optimal career choice for each $i$, $k$. Then for each $i$, calculate the average subjective expected utility from their new optimal career choice, and the ex post realized utility of that career. Also, for each $i$, calculate the share of graduates that chooses to switch careers, conditional on which career they chose in the first year. <br>

In [50]:
# write your answer here

## 3. <a id='toc3_'></a>[Problem 3: Barycentric interpolation](#toc0_)

**Problem:** We have a set of random points in the unit square,

$$
\mathcal{X} = \{(x_1,x_2)\,|\,x_1\sim\mathcal{U}(0,1),x_2\sim\mathcal{U}(0,1)\}.
$$

For these points, we know the value of some function $f(x_1,x_2)$,

$$
\mathcal{F} = \{f(x_1,x_2) \,|\, (x_1,x_2) \in \mathcal{X}\}.
$$

Now we want to approximate the value $f(y_1,y_2)$ for some  $y=(y_1,y_2)$, where $y_1\sim\mathcal{U}(0,1)$ and $y_2\sim\mathcal{U}(0,1)$.

**Building block I**

For an arbitrary triangle $ABC$ and a point $y$, define the so-called barycentric coordinates as:

$$
\begin{align*}
  r^{ABC}_1 &= \frac{(B_2-C_2)(y_1-C_1) + (C_1-B_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_2 &= \frac{(C_2-A_2)(y_1-C_1) + (A_1-C_1)(y_2-C_2)}{(B_2-C_2)(A_1-C_1) + (C_1-B_1)(A_2-C_2)} \\
  r^{ABC}_3 &= 1 - r_1 - r_2.
\end{align*}
$$

If $r^{ABC}_1 \in [0,1]$, $r^{ABC}_2 \in [0,1]$, and $r^{ABC}_3 \in [0,1]$, then the point is inside the triangle.

We always have $y = r^{ABC}_1 A + r^{ABC}_2 B + r^{ABC}_3 C$.

**Building block II**

Define the following points:

$$
\begin{align*}
A&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}>y_{2}\\
B&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}>y_{1}\text{ and }x_{2}<y_{2}\\
C&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}<y_{2}\\
D&=\arg\min_{(x_{1},x_{2})\in\mathcal{X}}\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}\text{ s.t. }x_{1}<y_{1}\text{ and }x_{2}>y_{2}.
\end{align*}
$$

**Algorithm:**

1. Compute $A$, $B$, $C$, and $D$. If not possible return `NaN`.
1. If $y$ is inside the triangle $ABC$ return $r^{ABC}_1 f(A) + r^{ABC}_2 f(B) + r^{ABC}_3 f(C)$.
1. If $y$ is inside the triangle $CDA$ return $r^{CDA}_1 f(C) + r^{CDA}_2 f(D) + r^{CDA}_3 f(A)$.
1. Return `NaN`.



**Sample:**

In [51]:
rng = np.random.default_rng(2024)

X = rng.uniform(size=(50,2))
y = rng.uniform(size=(2,))


**Questions 1:** Find $A$, $B$, $C$ and $D$. Illustrate these together with $X$, $y$ and the triangles $ABC$ and $CDA$.

In [52]:
# write your answer here

**Question 2:** Compute the barycentric coordinates of the point $y$ with respect to the triangles $ABC$ and $CDA$. Which triangle is $y$ located inside?

In [53]:
# write your answer here

Now consider the function:
$$
f(x_1,x_2) = x_1 \cdot x_2
$$

In [54]:
f = lambda x: x[0]*x[1]
F = np.array([f(x) for x in X])

**Question 3:** Compute the approximation of $f(y)$ using the full algorithm. Compare with the true value.

In [55]:
# write your answer here

**Question 4:** Repeat question 3 for all points in the set $Y$.

In [56]:
Y = [(0.2,0.2),(0.8,0.2),(0.8,0.8),(0.8,0.2),(0.5,0.5)]

In [57]:
# write your answer here